In [103]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 15.9 MB/s eta 0:00:00


In [107]:
from ultralytics import YOLO

model = YOLO("yolov8l.pt")

results = model("simin2.jpg")

for result in results:
  result.save(filename="result.jpg")


image 1/1 /content/simin2.jpg: 640x640 1 person, 1 bicycle, 1 motorcycle, 1 bottle, 2940.2ms
Speed: 4.4ms preprocess, 2940.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


In [105]:
model.export(format="onnx")

Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)
requirements: Ultralytics requirement ['onnxslim'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 4.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 13.7s, installed 1 package: ['onnxslim']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.39...
ONNX: export success ✅ 27.5s, saved as 'yolov8l.onnx' (166.8 MB)

Export complete (46.4s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8l.onnx'

In [ ]:
onnx_model = YOLO("yolov8l.onnx")

results = onnx_model("simin2.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...
Preferring ONNX Runtime AzureExecutionProvider

image 1/1 /content/simin2.jpg: 640x640 1 person, 1 bicycle, 1 motorcycle, 1 bottle, 2831.0ms
Speed: 3.2ms preprocess, 2831.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


In [108]:
import onnxruntime as ort
import numpy as np
from PIL import Image

# Load the ONNX model
session = ort.InferenceSession("yolov8l.onnx")

# Preprocess the image to fit the model's input requirements
def preprocess_image(image_path, input_size=(640, 640)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(input_size)
    image_data = np.array(image).astype('float32')
    image_data = np.transpose(image_data, (2, 0, 1))  # HWC to CHW
    image_data = np.expand_dims(image_data, axis=0)   # Add batch dimension
    image_data /= 255.0  # Normalize the image to [0, 1]
    return image_data

# Load and preprocess the input image
input_image = preprocess_image("simin2.jpg")

# Run inference
input_name = session.get_inputs()[0].name
outputs = session.run(None, {input_name: input_image})

# Post-process and handle the output as needed
print("Model outputs:", outputs)


Model outputs: [array([[[     14.031,      12.462,       29.26, ...,      529.56,      562.91,       598.7],
        [     5.7122,      5.7052,      6.3701, ...,      551.99,      549.05,      559.58],
        [     29.719,      26.996,      62.188, ...,      266.22,      170.91,      110.75],
        ...,
        [ 4.7684e-07,  2.9802e-07,  1.4901e-07, ...,  2.7716e-06,  2.8014e-06,  2.2352e-06],
        [ 2.6822e-07,  1.1921e-07,  1.1921e-07, ...,  2.6524e-06,  3.0994e-06,  3.2783e-06],
        [ 2.3842e-07,  1.4901e-07,  1.1921e-07, ...,   2.265e-06,  2.4736e-06,  2.4438e-06]]], dtype=float32)]


Scikit-Learn

In [49]:
!pip install onnxruntime

In [50]:
!pip install skl2onnx

In [109]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)

#model = RandomForestClassifier()
model = MLPClassifier(random_state=1, max_iter=300)
model.fit(X_train, y_train)


MLPClassifier(max_iter=300, random_state=1)

In [63]:
X[0]

array([5.1, 3.5, 1.4, 0.2], dtype=float32)

In [64]:
import skl2onnx

onnx_model = skl2onnx.to_onnx(model, X[0])
with open("scikit_learn_onnx_model.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())


In [110]:
import onnxruntime as ort

onnx_model = ort.InferenceSession("scikit_learn_onnx_model.onnx", providers=["CPUExecutionProvider"])



In [68]:
input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name
input_name, output_name

('X', 'output_label')

In [111]:
pred = onnx_model.run(
    [output_name],
    {input_name: X_test[0]})
pred

ValueError: Required inputs (['X']) are missing from input feed (['images']).

In [118]:
import numpy as np
import time
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import skl2onnx
import onnxruntime as ort

# Load and prepare data
iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train the model
model = MLPClassifier(random_state=1, max_iter=300)
model.fit(X_train, y_train)

# Convert to ONNX format
onnx_model = skl2onnx.to_onnx(model, X[0:1])
with open("scikit_learn_onnx_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

# Configure ONNX Runtime with custom session options
session_options = ort.SessionOptions()
session_options.intra_op_num_threads = 4  # Set the number of threads here

# Load the ONNX model with session options
onnx_model = ort.InferenceSession("scikit_learn_onnx_model.onnx", sess_options=session_options, providers=["CPUExecutionProvider"])

# Get input and output names
input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name

# Measure inference time
start_time = time.time()
pred = onnx_model.run([output_name], {input_name: X_test[0:1]})
end_time = time.time()

# Print results and execution time
print("Prediction:", pred)
print("Inference time:", end_time - start_time, "seconds")


Prediction: [array([1], dtype=int64)]
Inference time: 0.00024509429931640625 seconds


PyTorch

In [26]:
!pip install onnx
!pip install onnxscript


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [74]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_model = torch.onnx.dynamo_export(torch_model, torch_input)

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/_exporter_legacy.py:116: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [75]:
torch_input

tensor([[[[-0.6344,  1.1651, -0.1903,  ..., -1.5320,  0.2077, -0.9310],
          [ 1.1036,  0.0306,  0.8584,  ...,  0.2988,  1.0188, -0.2341],
          [ 1.2444,  1.4178, -0.8850,  ...,  1.3727, -0.3602, -0.2984],
          ...,
          [ 0.0837,  0.1616,  2.0458,  ..., -0.0372,  0.0589, -0.1906],
          [-1.4916,  0.2706,  0.2351,  ..., -0.2213, -0.7154,  0.8151],
          [ 0.4400,  0.8115, -1.2699,  ...,  0.0347,  1.8701, -0.7620]]]])

In [76]:
onnx_model.save("my_image_classifier.onnx")

In [77]:
import onnx
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

In [78]:
import onnxruntime as ort

onnx_model = ort.InferenceSession("my_image_classifier.onnx", providers=["CPUExecutionProvider"])

input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name
input_name, output_name

onnx_input = torch_input.detach().cpu().numpy()


pred = onnx_model.run(
    [output_name],
    {input_name: onnx_input }
)

pred

[array([[-0.01943916,  0.02073416, -0.208087  , -0.05055603, -0.11559197,
         -0.08344416, -0.09249496,  0.0510811 , -0.06643267,  0.09465813]],
       dtype=float32)]

In [81]:
onnxruntime_outputs = pred
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[-0.01943916,  0.02073416, -0.208087  , -0.05055603, -0.11559197,
        -0.08344416, -0.09249496,  0.0510811 , -0.06643267,  0.09465813]],
      dtype=float32)]


TensorFlow

In [91]:
!pip install tf2onnx

In [92]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50

model = ResNet50(weights="imagenet", include_top=True)


In [93]:
import tf2onnx

input_data = [tf.TensorSpec(
        model.inputs[0].shape,
        dtype=model.inputs[0].dtype,
        name=model.inputs[0].name
    )]

onnx_model, _ = tf2onnx.convert.from_keras(
    model, input_data)


In [94]:
onnx.save(onnx_model, "resnet50.onnx")

In [119]:
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 1.5 MB/s eta 0:00:00


In [102]:
import onnxruntime as ort
import numpy as np
from PIL import Image
from tensorflow.keras.applications.resnet50 import decode_predictions



onnx_model = ort.InferenceSession("resnet50.onnx", providers=["CPUExecutionProvider"])

input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name
input_name, output_name

# onnx_input = np.random.normal(size=[1, 224, 224, 3]).astype(np.float32)
image = Image.open("bear.jpg")
image = image.resize((224, 224))
image = np.array(image).astype(np.float32)
image = np.expand_dims(image, axis=0)
onnx_input = image

pred = onnx_model.run(
    [output_name],
    {input_name: onnx_input }
)

decode_predictions(pred[0])

[[('n02132136', 'brown_bear', 0.41550723),
  ('n02134084', 'ice_bear', 0.2602609),
  ('n02133161', 'American_black_bear', 0.09963104),
  ('n01883070', 'wombat', 0.025618915),
  ('n02134418', 'sloth_bear', 0.019697057)]]

In [1]:
import onnxruntime as ort
import numpy as np
from PIL import Image
from tensorflow.keras.applications.resnet50 import decode_predictions

# Load the ONNX model with GPU support
onnx_model = ort.InferenceSession("resnet50.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

# Prepare the input image
image = Image.open("bear.jpg")
image = image.resize((224, 224))
image = np.array(image).astype(np.float32)
image = np.expand_dims(image, axis=0)

# Run inference
input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name

pred = onnx_model.run([output_name], {input_name: image})

# Decode predictions
print(decode_predictions(pred[0]))


[[('n02132136', 'brown_bear', 0.41550723), ('n02134084', 'ice_bear', 0.2602609), ('n02133161', 'American_black_bear', 0.09963104), ('n01883070', 'wombat', 0.025618915), ('n02134418', 'sloth_bear', 0.019697057)]]


TORCH

In [9]:
!pip install torch torchvision onnx onnx-simplifier

In [14]:
import torch
import torch.onnx

# Example: Load a pre-trained model (e.g., a ResNet)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

# Create a dummy input tensor with the required input shape (e.g., batch size 1, 3 color channels, 224x224 image)
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(
    model,               # PyTorch model
    dummy_input,         # Dummy input for the model
    "model.onnx",        # Output file name
    opset_version=11,    # ONNX opset version
    input_names=['input'],  # Input tensor name
    output_names=['output'], # Output tensor name
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # Allow dynamic batch size
)

print("Model has been successfully converted to ONNX format.")


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Model has been successfully converted to ONNX format.


SCRFD

In [55]:
!pip install mmdet
!pip install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.1/479.1 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv: filename=mmcv-2.2.0-cp310-cp310-linux_x86_64.whl size=28696974 sha256=5fb475a20a268e5ec5865bba2d633cb1e173e64295a66b72e6e4b282f0208ffb
  Stored in directory: /root/.cache/pip/wheels/dd/50/9f/4e0c45c952bddd25de1cae1b310bd1b20dd69967b5202ca1af
Successfully built mmcv


In [56]:
!python scrfd2onnx.py base_500m.py model.pth --output-file output.onnx --shape 640 640


Traceback (most recent call last):
  File "/content/scrfd2onnx.py", line 10, in <module>
    from mmdet.core import (build_model_from_cfg, generate_inputs_and_wrap_model,
  File "/usr/local/lib/python3.10/dist-packages/mmdet/__init__.py", line 25, in <module>
    and mmcv_version <= digit_version(mmcv_maximum_version)), \
AssertionError: MMCV==2.2.0 is used but incompatible. Please install mmcv>=1.3.17, <=1.6.0.
